<a href="https://colab.research.google.com/github/rallyfranky/my-first-repo/blob/main/signate_comp_Lgb_bayesOpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('/content/drive/MyDrive/signate/train.csv')
test = pd.read_csv('/content/drive/MyDrive/signate/test .csv')

In [3]:
train.head()

,gameId,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueTotalGold,blueTotalExperience,blueWins
0,0,0,5,8,6,0,0,14536,17256,0
1,1,1,10,1,5,0,0,14536,17863,0
2,2,0,3,10,2,0,0,17409,17256,0
3,3,1,7,10,8,0,0,19558,18201,0
4,4,0,4,9,4,0,0,17409,17256,0


In [4]:
train_x = train.drop('blueWins', axis=1)
train_y = train['blueWins']

In [5]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_score

In [6]:
!pip install optuna
import optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.5/348.5 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 KB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.1/147.1 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 KB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 KB 15.3 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=8c7eca403d9430d0965af4439e2791554b6c15f85e2eccb638f87ee418f6a5c0
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip
  

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

train_set, test_set = train_test_split(train, test_size=0.2, random_state=0)
X_train = train_set.drop('blueWins', axis=1)
y_train = train_set['blueWins']
X_test = test_set.drop('blueWins', axis=1)
y_test = test_set['blueWins']

In [16]:
def objective(trial):
#    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.25)
    dtrain = lgb.Dataset(X_train, label=y_train)
 
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }
 
    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(X_test)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(y_test, pred_labels)
    return accuracy

In [17]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)
 
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2023-01-15 03:07:00,039] A new study created in memory with name: no-name-0b2d891a-73d1-4b05-898a-6f5e3456d688
<ipython-input-16-b9065a5aeb31>:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
<ipython-input-16-b9065a5aeb31>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
<ipython-input-16-b9065a5aeb31>:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instea

Number of finished trials: 100
Best trial: {'lambda_l1': 0.0006982817739140654, 'lambda_l2': 0.00020414685333177673, 'num_leaves': 223, 'feature_fraction': 0.7738959829889188, 'bagging_fraction': 0.7242453626272797, 'bagging_freq': 1, 'min_child_samples': 91}


In [18]:
print('Best score is{}'.format(study.best_value))

Best score is0.816875


In [19]:
test.head()

,gameId,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueTotalGold,blueTotalExperience
0,9,0,7,6,6,0,0,16961,18201
1,15,0,6,6,6,2,1,18513,18021
2,18,1,6,4,3,0,0,13475,17256
3,23,0,5,4,7,0,0,17409,17256
4,31,0,10,8,9,0,0,18117,18472


In [22]:
param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'lambda_l1': 0.0006982817739140654,
        'lambda_l2': 0.00020414685333177673,
        'num_leaves': 223,
        'feature_fraction': 0.7738959829889188,
        'bagging_fraction': 0.7242453626272797,
        'bagging_freq': 1,
        'min_child_samples': 91,
    }
dtrain = lgb.Dataset(X_train, label=y_train)
model = lgb.train(param, dtrain)
test['pred'] = model.predict(test)

In [25]:
test['pred'] = np.rint(test['pred'])

In [26]:
test

,gameId,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueTotalGold,blueTotalExperience,pred
0,9,0,7,6,6,0,0,16961,18201,1.0
1,15,0,6,6,6,2,1,18513,18021,1.0
2,18,1,6,4,3,0,0,13475,17256,0.0
3,23,0,5,4,7,0,0,17409,17256,0.0
4,31,0,10,8,9,0,0,18117,18472,1.0
...,...,...,...,...,...,...,...,...,...,...
1995,9971,0,6,3,7,0,0,17409,17256,0.0
1996,9980,0,4,4,4,0,0,17409,17256,0.0
1997,9983,1,6,3,3,1,1,18513,18201,1.0
1998,9996,1,10,9,9,1,1,18513,18201,1.0


In [28]:
submit = test[['gameId','pred']]

In [29]:
submit.to_csv('submit.csv')